### Propagation of Uncertainty Equation:

\begin{equation}
Var(f) = \sum^N_{i=1} \left[ \left(\frac{\partial f}{\partial x_i} \right)^2 Var(x_i) \right] + 2 \frac{\partial f}{\partial x_1}\frac{\partial f}{\partial x_2} Cov(x_1, x_2) + 2 \frac{\partial f}{\partial x_2}\frac{\partial f}{\partial x_3} Cov(x_2, x_3) + ...,
\end{equation}

$Uncertainty(f) = \sqrt{Var(f)}$



where $f$ is some function. In this case, $f$ is the equation for $E_{pk}$. From the Band function, 
\begin{equation}
E_{pk} = (\alpha + 2) E_{0} ,
\end{equation}
where $E_0$ is the e-folding energy with a parameter name of 'tem'. 

$E_{pk}$ - epeak

$E_{0}$ - tem

$\alpha$ - alpha, or low-energy index.

$\beta$ - beta, or high-energy index.

$Amp$ - model amplitude (the 'norm' parameter). 

The partial derivatives of $E_{pk}$ are:
\begin{equation}
\begin{split}
\frac{\partial E_{pk}}{\partial \alpha} &= E_0 \\
\frac{\partial E_{pk}}{\partial \beta} &= 0 \\
\frac{\partial E_{pk}}{\partial E_0} &= \alpha + 2\\
\frac{\partial E_{pk}}{\partial Amp} &= 0
\end{split}
\end{equation}

Since the model amplitude and $\beta$ are not even in the $E_{pk}$ equation, there really is no reason to take the partial derivatives of the equation with respect to them. However, if we do so, we can keep all the terms in the variance-covariance matrix as is. This is advantageous for using the matrix form of the propagation equation. Otherwise, the partial derivative matrix and the variance-covariance matrix will have different shapes. 

The partial derivative matrix:
\begin{bmatrix}
\frac{\partial E_{pk}}{\partial \alpha} & \frac{\partial E_{pk}}{\partial \beta} & \frac{\partial E_{pk}}{\partial E_0} & \frac{\partial E_{pk}}{\partial Amp}
\end{bmatrix}


The variance-covariance matrix:

\begin{bmatrix}
Var(x_1,x_1) & Cov(x_1,x_2) & Cov(x_1,x_3) & Cov(x_1,x_4) \\
Cov(x_2,x_1) & Var(x_2,x_2) & Cov(x_2,x_3) & Cov(x_2,x_4) \\
Cov(x_3,x_1) & Cov(x_3,x_2) & Var(x_3,x_3) & Cov(x_3,x_4) \\
Cov(x_4,x_1) & Cov(x_4,x_2) & Cov(x_4,x_3) & Var(x_4,x_4) \\  
\end{bmatrix}


and with the proper terms:


\begin{bmatrix}
Var(\alpha,\alpha) & Cov(\alpha,\beta) & Cov(\alpha,E_0) & Cov(\alpha,Amp) \\
Cov(\beta,\alpha) & Var(\beta,\beta) & Cov(\beta,E_0) & Cov(\beta,Amp) \\
Cov(E_0,\alpha) & Cov(E_0,\beta) & Var(E_0,E_0) & Cov(E_0,Amp) \\
Cov(Amp,\alpha) & Cov(Amp,\beta) & Cov(Amp,E_0) & Var(Amp,Amp) \\
\end{bmatrix}

The above propagation of uncertainty equation can be computed by matrix algebra:

$Uncertainty = \sqrt{PD * COV * PD^T}$
where PD is the partial derivatives matrix and COV is the variance-covariance matrix from the model fit. 

In [1]:
from __future__ import division
from collections import OrderedDict
import os
import json

from math import atanh, log
import numpy as np
import pandas as pd
from astropy.io import fits as pyfits

In [2]:
def epeak_partials(model):
    """

    Parameters:
    -----------
    model: str, model name. 
            Options are: 'grbm', 'cutoffpl', 'sbpl'
            THESE ARE XSPEC ONLY!
    dpar: str or None, default is None.
            Pass None and get a dictionary of all the partial derivatives.
            Pass a single string and get only the partial derivative
              wrt that parameter. 

    Notes:
    ------
    The 'sbpl' model uses brkscale = 0.3. If you don't want to fix it 
    at this value, update this function and add brkscale to the parameters.
    We currently don't allow the function to use this parameter, so for 
    now it doesn't matter. 
    
    *********
    Propagating ebreak and tem errors to find epeak's error uses the same 
    propragation of error equation as any function.
    The matrix version of the equation is: 
    variance = PDMAT * COVARMAT * PDMAT.T
    uncertainty = np.sqrt(variance)
    Where PDMAT is the partial derivative matrix and COVARMAT is the 
    covariance matrix from the fit. 
    The function for computing epeak from ebreak or tem does not include 
    the amplitude term (i.e., norm__4 parameter), therefore there is no 
    apparent reason for including its partial derivative (which is 0). 
    However, we include it here so that the covariance matrix from the 
    model fit does not need to be changed. If we do edit the covariance 
    matrix and remove the norm__4 terms (which is row 3 and col 3 for 
    sbpl and grbm fits) we would get the same exact result as if we left 
    it in and had the partial deriv wrt norm as 0.

    """
    pars = OrderedDict([
                ('alpha__1', 
                    'tem__3'), 
                ('beta__2', 
                    '0'), 
                ('tem__3', 
                    'alpha__1 + 2.0'),
                ('norm__4', 
                    '0')
                ])
    return pars


In [3]:
def delete_pars(parameters):
    """
    Parameters:
    ------------
    parameters: dict of parameters and values. 
                parameters.keys() are the parameter names;
                'alpha__1', 'beta__2', etc.

    Clears all global parameter assignments.
    del alpha__1
    del beta__2
    del tem__3 
    etc ...

    """
    for key in parameters.keys():
        del globals()[key]


def Calc_Epeak_Uncertainty(model, parameters, covarmat):
    """
    Calculates Epeak Uncertainty.

    Parameters:
    -----------
    model: str, model. If you have 'grbm' with 'tem', this gives you
                uncertainty in epeak. Need to compute epeak elsewhere. 

    parameters: ordered dict, holds parameter names associated with 
                the respective model. See bottom for examples. 
    covarmat: np.ndarray or np.matrix, 
                square covariance matrix from XSPEC model fit. 
                Needs to be read from a FITS file. 

    Notes:
    ------
    The function for calculating epeak from ebreak or tem does not 
    include the model amplitude term (i.e., norm__4 parameter). 
    Therefore it doesn't make sense to include a partial derivative of 
    the fn wrt this parameter. HOWEVER, we did this, which is 0, so that 
    we can leave the covariance matrix from the fit in its present form. 
    If we removed the pd wrt norm__4, then we'd have to remove all the 
    norm__4 terms in the covariance matrix (which is all of row 3 
    and col 3 for sbpl and grbm fits - starting at 0). Without the 
    norm__4 in the partial derivative matrix, the shapes between the 
    pd matrix and covariance matrix will be different.  
    We tested it, and if we remove the norm__4 terms from both the 
    pd matrix and the covariance matrix, we will get the same thing if 
    we just leave the terms in both. 


    """

    if isinstance(parameters, dict) is False:
        raise Exception, "parameters must be a dictionary."
    # Dictionary holding str eqns for the partial derivatives of the fn wrt each param. 
    Partials = eval("epeak_partials('%s')"%model) # call to epeak_partials fn

    # Instantiate only appropriate parameters. 
    #  replaced parameteres.keys() with Partials.keys()
    for key in Partials.keys():
        globals()[key] = parameters[key]
    
    # Calculate Epeak
    epeak = (alpha__1 + 2.0) * tem__3
    
    # Evaluate partial derivs with real numbers and store in matrix. 
    pdmatrix = np.asmatrix([eval(i) for i in Partials.values()]) 
    
    # Propagation of Uncertainty Calculation.
    variance = pdmatrix * covarmat * pdmatrix.T
    uncertainty = float(np.sqrt(variance))

    delete_pars(parameters) # clears global def of alpha__1, beta__2, ...
    
    return epeak, uncertainty

## Begin Program

In [4]:
burst_data = pd.read_csv('/Users/KimiZ/GRBs2/Sample/bursts.txt', 
                         sep=' ', header=0)
burst_data['duration'] = burst_data.t90_stop-burst_data.t90_start
burst_data.head()

,number,name,trigger,z,t90_start,t90_stop,duration
0,1,GRB080804,bn080804972,2.2045,0.256,24.960,24.704
1,2,GRB080810,bn080810549,3.3604,-20.096,87.361,107.457
2,3,GRB080905A,bn080905499,0.1218,-0.064,0.896,0.960
3,4,GRB080905B,bn080905705,2.3739,-5.120,100.864,105.984
4,5,GRB080916A,bn080916406,0.6887,0.512,46.849,46.337


In [5]:
model      = 'grbm'
det        = 'L'             # detector fit:  'L' stands for GBM+LLE+LAT
version    = '-01-'          # version of fit: first version. 


# Partial Derivatives of the epeak function wrt each parameter. 
#   There is no norm__4 in the epeak calculation, but we leave it as 0 
#   so that we can multiply the partial derivative matrix by the 
#   covariance matrix. They need to have the same shape. 
Partials = epeak_partials(model=model)
Partials

OrderedDict([('alpha__1', 'tem__3'),
             ('beta__2', '0'),
             ('tem__3', 'alpha__1 + 2.0'),
             ('norm__4', '0')])

In [6]:
filename   = 'AmatiOverlap_38_bothdatasets.txt'

filename = '/Users/KimiZ/GRBs2/Sample/'+filename
#df = pd.read_csv(filename, sep='\t')
df = pd.read_csv(filename, sep=',')

In [7]:
filename

'/Users/KimiZ/GRBs2/Sample/AmatiOverlap_38_bothdatasets.txt'

In [9]:
df.head()

,trigger,name,number,z,LumDist,trigtime,t90,t90Rest,t90start,t90stop,...,Flue2,Flue2_err,Flue3,Flue3_err,Eiso1,Eiso1_err,Eiso2,Eiso2_err,Eiso3,Eiso3_err
0,bn080804972,80804,0,2.2045,5.536738e+28,54682.97239,24.704,7.709159,0.256,24.960,...,0.000010,0.000002,0.000010,1.743566e-06,1.159516e+53,2.096977e+52,1.165993e+53,2.687305e+52,1.163744e+53,2.096022e+52
1,bn080810549,80810,1,3.3604,9.206175e+28,54688.54876,107.456,24.643611,-20.096,87.360,...,0.000031,0.000010,0.000027,1.885465e-06,6.407256e+53,4.536912e+52,7.456679e+53,2.533761e+53,6.508130e+53,4.605327e+52
2,bn080916009,080916C,5,4.3500,1.250788e+29,54725.00886,62.977,11.771402,1.280,64.257,...,0.000191,0.000004,0.000095,1.931454e-06,3.482210e+54,7.055090e+52,7.033527e+54,1.472875e+53,3.494756e+54,7.098348e+52
3,bn080928628,80928,6,1.6919,4.006108e+28,54737.62843,14.336,5.325606,-1.792,12.544,...,0.000002,0.000001,0.000002,5.836189e-07,1.377612e+52,4.425007e+51,1.480658e+52,1.018460e+52,1.609213e+52,4.372458e+51
4,bn081008832,81008,7,1.9670,4.818115e+28,54747.83197,150.016,50.561510,0.000,150.016,...,0.000113,0.000037,0.000017,1.517899e-06,1.698696e+53,1.470818e+52,1.109155e+54,3.655387e+53,1.707400e+53,1.492415e+52


In [ ]:
df

In [10]:
bursts = '''bn080916009
bn090323002
bn090328401
bn090510016
bn090902462
bn090926181
bn091003191
bn091208410
bn100728095
bn110731465
bn130518580
bn131108862
bn131231198'''
bursts = bursts.split('\n')

In [11]:
kim = df.copy()

In [12]:
for burst in bursts:
    z = float(burst_data.loc[burst_data['trigger'] == burst].z)
    dur = float(burst_data.loc[burst_data['trigger'] == burst].duration)    
    
    # *~*~*~ Find appropriate files.
    detDir = ('GBMwLAT' if 'L' in det else 'GBM')
    paramfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.json'%(burst, detDir, model, model, 
                                                          version, det))
    covarfile = ('/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/'
                 '%s/%s/xspec_fitresults_%s_%s_%s_.fit'%(burst, detDir, model, model, 
                                                         version, det))
    
    # *~*~*~ Covaraince Matrix
    f1 = pyfits.open(covarfile)
    COVARMAT = np.asarray(f1[2].data['COVARMAT'][0])
    nPars = int(np.sqrt(len(COVARMAT)))
    COVARMAT = COVARMAT.reshape((nPars, nPars))
    f1.close()
    del f1
    
    # *~*~*~ parameters dict -- object_pairs_hook=OrderedDict keeps it in order. 
    Parameters = json.load(open(paramfile, 'r'), 
                           object_pairs_hook=OrderedDict)[0] 
    
    # Keep only the param value, not the errors. 
    for key in Parameters.keys():
        Parameters[key] = Parameters[key][0]
        
    epeak, epeakUnc = Calc_Epeak_Uncertainty(model=model, 
                                             parameters=Parameters, 
                                             covarmat=COVARMAT)
    print('%s:  %10.5f    +-%10.5f'%(burst, epeak, epeakUnc))
    df.loc[df.trigger == burst, 'epeak_err_low_mine'] = epeakUnc
    df.loc[df.trigger == burst, 'epeak_err_up_mine'] = epeakUnc
    df.loc[df.trigger == burst, 'epeakRest_err_low_mine'] = epeakUnc * (1+z)
    df.loc[df.trigger == burst, 'epeakRest_err_up_mine'] = epeakUnc * (1+z)

bn080916009:   510.64139    +-  34.34517
bn090323002:   702.28847    +-  53.21646
bn090328401:   635.22789    +-  51.79216
bn090510016:  2052.27913    +- 331.06326
bn090902462:  1020.81855    +-  18.39910
bn090926181:   302.71071    +-   3.96333
bn091003191:   387.65164    +-  23.44402
bn091208410:   104.26595    +-  17.67836
bn100728095:   298.07698    +-   9.75014
bn110731465:   256.50378    +-  14.05121
bn130518580:   422.06720    +-  14.74287
bn131108862:   330.41515    +-  17.01770
bn131231198:   220.39260    +-   3.39933


In [20]:
df.loc[df.LATburst == 'Y'].epeakRest_err_low_mine

2     183.746653
9     243.199227
10     89.911198
14     51.922253
15     12.310902
17     44.470956
21     36.475764
30     53.816136
Name: epeakRest_err_low_mine, dtype: float64

In [21]:
df.loc[df.LATburst == 'Y'].epeakRest_err_up_mine

2     183.746653
9     243.199227
10     89.911198
14     51.922253
15     12.310902
17     44.470956
21     36.475764
30     53.816136
Name: epeakRest_err_up_mine, dtype: float64

In [23]:
df.to_csv(filename, sep='\t', index=False)

In [ ]:
filename

In [ ]:
df1 = pd.read_csv(filename, sep='\t')

In [ ]:
df1.head()

### epeak_err_low, epeak_err_up, epeakRest_err_low, epeakRest_err_up

In [ ]:
df.number.tolist()

In [24]:
df.epeak_err_low_mine.to_clipboard(index=False) # epeak_err_low

In [25]:
df.epeak_err_up_mine.to_clipboard(index=False) # epeak_err_up

In [26]:
df.epeakRest_err_low_mine.to_clipboard(index=False) # epeakRest_err_low

In [27]:
df.epeakRest_err_up_mine.to_clipboard(index=False) # epeakRest_err_up

### files that have already been updated:

- Band_13_GBM+LAT__22_GBMconstrained.txt
